# Performance Analysis: local_asymmetric_sigclip Function

This notebook analyzes the performance bottlenecks in the `local_asymmetric_sigclip` function to understand why the optimization didn't provide the expected speedup.

## 1. Import Required Libraries

Import necessary libraries for performance analysis including time, cProfile, memory_profiler, and any relevant modules.

In [6]:
import numpy as np
import time
import cProfile
import pstats
from io import StringIO
from astropy.stats import sigma_clip

# Import your continuum normalization module
from cont_norm import local_asymmetric_sigclip

# For memory profiling (install with: pip install memory_profiler)
# %load_ext memory_profiler

print("Libraries imported successfully!")

Libraries imported successfully!


## Create Test Data

Generate synthetic spectrum data similar to your real data to isolate performance issues.

In [8]:
spectrum_index = 0  # Change this index to plot a different spectrum
from hdf5_spectrum_reader import HDF5SpectrumReader

# Load the spectrum data
spectrum_file = "../data/weave_nlte_grids.h5"
reader = HDF5SpectrumReader(spectrum_file)

wavelength, flux = reader.get_wavelength_flux(spectrum_index)

## 2. Measure Execution Time with %timeit

Use Jupyter's %timeit magic command to get accurate timing measurements.

In [9]:
# Test with your original function
print("Testing local_asymmetric_sigclip performance...")
%timeit -n 1 -r 1 result = local_asymmetric_sigclip(flux, wavelength, window_width=10.0)

Testing local_asymmetric_sigclip performance...
3min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 3. Profile Code with %prun

Use %prun magic command to identify which functions and lines of code are consuming the most time.

In [10]:
# Profile the function to see where time is spent
%prun -l 20 result = local_asymmetric_sigclip(flux, wavelength, window_width=10.0)

         206446619 function calls (205164932 primitive calls) in 251.698 seconds

   Ordered by: internal time
   List reduced from 113 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1281687   28.063    0.000  113.586    0.000 nanfunctions.py:1617(nanvar)
  8119990   20.035    0.000   20.035    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  2563374   16.342    0.000   38.703    0.000 nanfunctions.py:187(_divide_by_count)
   107467   15.595    0.000  237.255    0.002 sigma_clipping.py:429(_sigmaclip_noaxis)
  1389154    9.663    0.000   18.766    0.000 _methods.py:101(_mean)
  1389154    9.382    0.000   58.932    0.000 function_base.py:3931(_median)
  1389154    9.380    0.000    9.380    0.000 {method 'partition' of 'numpy.ndarray' objects}
  1281687    8.826    0.000  214.376    0.000 sigma_clipping.py:313(_compute_bounds)
  5341682    8.295    0.000   18.159    0.000 _ufunc_config.py:33(seterr)
  5234215    8.037    0.000

## 4. Detailed Component Analysis

Let's analyze each component of the function separately to identify the bottleneck.

In [11]:
# Test individual components to identify the bottleneck
def test_searchsorted_performance():
    """Test just the window finding part"""
    n = len(flux)
    times = []
    
    start = time.time()
    for i in range(n):
        wl_center = wavelength[i]
        wl_min = wl_center - 5.0  # 10 Å window
        wl_max = wl_center + 5.0
        
        left_idx = np.searchsorted(wavelength, wl_min, side='left')
        right_idx = np.searchsorted(wavelength, wl_max, side='right')
    
    end = time.time()
    return end - start

def test_sigma_clip_performance():
    """Test sigma clipping on typical window sizes"""
    # Typical window size in your spectra
    typical_window_size = int(10 * n_points / (wavelength.max() - wavelength.min()))
    print(f"Typical window size: {typical_window_size} points")
    
    # Sample some windows
    n_samples = 1000
    times = []
    
    start = time.time()
    for i in range(n_samples):
        start_idx = np.random.randint(0, n_points - typical_window_size)
        window_flux = flux[start_idx:start_idx + typical_window_size]
        
        res = sigma_clip(window_flux, sigma_lower=0.5, sigma_upper=2.0, maxiters=None)
        clip_vals = res.compressed()
        if clip_vals.size > 0:
            loc = np.median(clip_vals)
        else:
            loc = np.median(window_flux)
    
    end = time.time()
    return end - start

# Run component tests
print("Testing searchsorted performance...")
searchsorted_time = test_searchsorted_performance()
print(f"Searchsorted time for {n_points} iterations: {searchsorted_time:.2f} seconds")

print("\nTesting sigma_clip performance...")
sigclip_time = test_sigma_clip_performance()
print(f"Sigma clipping time for 1000 samples: {sigclip_time:.2f} seconds")
print(f"Estimated sigma clipping time for {n_points} iterations: {sigclip_time * n_points / 1000:.2f} seconds")

Testing searchsorted performance...
Searchsorted time for 50000 iterations: 5.57 seconds

Testing sigma_clip performance...
Typical window size: 912 points
Sigma clipping time for 1000 samples: 1.48 seconds
Estimated sigma clipping time for 50000 iterations: 74.20 seconds


## 5. Alternative Optimization Approaches

Based on the bottleneck analysis, let's implement more aggressive optimizations.

In [12]:
def local_asymmetric_sigclip_optimized_v2(flux, wavelength, window_width=10.0, sigma_lower=0.5, sigma_upper=2.0):
    """
    Heavily optimized version with reduced sigma_clip calls and vectorization where possible.
    """
    flux = np.asarray(flux)
    wavelength = np.asarray(wavelength)
    n = len(flux)
    norm = np.empty_like(flux)
    
    # Pre-compute window size in pixels (approximate)
    delta_wl = np.median(np.diff(wavelength))
    window_pixels = int(window_width / delta_wl)
    
    # Use a simpler sliding window approach with fixed pixel window
    for i in range(n):
        # Use fixed pixel window centered on current point
        left_idx = max(0, i - window_pixels // 2)
        right_idx = min(n, i + window_pixels // 2 + 1)
        
        window_flux = flux[left_idx:right_idx]
        
        if window_flux.size == 0:
            norm[i] = flux[i]
            continue
            
        # Quick outlier rejection without full sigma_clip
        # Use percentile-based clipping instead
        if window_flux.size > 5:  # Only if we have enough points
            p_low = np.percentile(window_flux, 15.87)  # ~1 sigma below
            p_high = np.percentile(window_flux, 84.13)  # ~1 sigma above
            
            # Apply asymmetric clipping
            mask_low = window_flux >= (np.median(window_flux) - sigma_lower * (np.median(window_flux) - p_low))
            mask_high = window_flux <= (np.median(window_flux) + sigma_upper * (p_high - np.median(window_flux)))
            
            clipped_flux = window_flux[mask_low & mask_high]
            
            if clipped_flux.size > 0:
                loc = np.median(clipped_flux)
            else:
                loc = np.median(window_flux)
        else:
            loc = np.median(window_flux)
            
        norm[i] = flux[i] / loc if loc != 0 else flux[i]
    
    return norm

def local_asymmetric_sigclip_super_fast(flux, wavelength, window_width=10.0, sigma_lower=0.5, sigma_upper=2.0, stride=10):
    """
    Ultra-fast version: compute continuum at sparse grid points and interpolate.
    """
    flux = np.asarray(flux)
    wavelength = np.asarray(wavelength)
    n = len(flux)
    
    # Compute continuum estimates at every 'stride' points
    sparse_indices = np.arange(0, n, stride)
    sparse_continuum = np.empty(len(sparse_indices))
    
    delta_wl = np.median(np.diff(wavelength))
    window_pixels = int(window_width / delta_wl)
    
    for j, i in enumerate(sparse_indices):
        left_idx = max(0, i - window_pixels // 2)
        right_idx = min(n, i + window_pixels // 2 + 1)
        
        window_flux = flux[left_idx:right_idx]
        
        # Simple robust estimator
        if window_flux.size > 3:
            sorted_flux = np.sort(window_flux)
            # Use interquartile range for robust estimate
            q1_idx = len(sorted_flux) // 4
            q3_idx = 3 * len(sorted_flux) // 4
            sparse_continuum[j] = np.mean(sorted_flux[q1_idx:q3_idx])
        else:
            sparse_continuum[j] = np.median(window_flux) if window_flux.size > 0 else flux[i]
    
    # Interpolate continuum to all points
    continuum = np.interp(np.arange(n), sparse_indices, sparse_continuum)
    
    # Normalize
    continuum = np.where(continuum == 0, 1, continuum)
    norm = flux / continuum
    
    return norm

print("Optimized functions defined!")

Optimized functions defined!


## 6. Compare Performance of Different Approaches

In [14]:
# Compare performance of different approaches
print("Performance Comparison:")
print("=" * 50)

#print("\n1. Original optimized function (searchsorted):")
#%timeit -n 1 -r 1 result1 = local_asymmetric_sigclip(flux, wavelength, window_width=10.0)

print("\n2. V2 Optimized (percentile-based clipping):")
%timeit -n 1 -r 3 result2 = local_asymmetric_sigclip_optimized_v2(flux, wavelength, window_width=10.0)

print("\n3. Super Fast (sparse grid + interpolation):")
%timeit -n 1 -r 5 result3 = local_asymmetric_sigclip_super_fast(flux, wavelength, window_width=10.0, stride=50)

Performance Comparison:

2. V2 Optimized (percentile-based clipping):
40.6 s ± 54.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

3. Super Fast (sparse grid + interpolation):
169 ms ± 799 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


## Key Insights and Recommendations

Based on the profiling results, the main bottleneck is likely the **sigma_clip function** being called 50,000 times (once per pixel). Each call to `sigma_clip` has significant overhead even for small arrays.

**The real performance killer is:**
1. **Function call overhead**: `sigma_clip` is called n times
2. **Statistical computation overhead**: Mean, std calculation for each small window  
3. **Masking operations**: Creating and applying masks for each window

**Solutions to try:**
1. Replace `sigma_clip` with simple percentile-based outlier rejection
2. Use sparse sampling + interpolation for massive speedup
3. Consider chunked processing or parallel processing
4. Pre-compute statistics where possible

Run this notebook with your actual data to see which approach works best for your use case!